# CS 6476 Computer Vision: VIVIT Lie Detector Project

In [1]:
!pip -q install einops
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Access is denied.


In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
if os.getcwd() != '/content/drive/My Drive/ViViT_Lie_Detector':
  os.chdir('drive/MyDrive/ViViT_Lie_Detector')
print(os.getcwd())

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import torch
from torch import nn, einsum
import torch.nn.functional as F
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from module import Attention, PreNorm, FeedForward
import numpy as np
from math import ceil
import argparse
from CONSTANTS import BATCH_SIZE, PATCH_SIZE, IMG_SIZE, FRAME_NUM
import zipfile
from tqdm import tqdm
import io
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from vivit import ViViT, Transformer
from Trainer import VIVIT_Trainer
import matplotlib.pyplot as plt
import cv2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'using {device} device...')

using cpu device...


### Loading Processed Dataset

In [ ]:
w = 480
h = 640
c = 3

img_dataset = []
''' _old/stack for midterm/final '''
# img_archive = zipfile.ZipFile('Processing_results/filtered_image_stack_old.zip', 'r')
img_archive = zipfile.ZipFile('Processing_results/filtered_image_stack.zip', 'r')

for file in tqdm(img_archive.infolist()):
  if file.filename.endswith('.npy'):
    with img_archive.open(file) as npy_file:
      img = np.load(io.BytesIO(npy_file.read()))
      if img.shape[1] == w and img.shape[2] == h and img.shape[3] == c:
        img_dataset.append(img)
print(len(img_dataset))

100%|██████████| 957/957 [03:51<00:00,  4.14it/s]

318


In [ ]:
# find max t
t_max = 0

for img in img_dataset:
  t_max = max(t_max, img.shape[0])
  # ensure unified frames
  # assert img.shape[1] == w and img.shape[2] == h and img.shape[3] == c
  if not (img.shape[1] == w and img.shape[2] == h and img.shape[3] == c):
    print("inconsistent shape: ",img.shape)
print(t_max)

120


In [ ]:
# ImageDataset
img_tensors = []
for img in tqdm(img_dataset):
  img_tensors.append(torch.tensor(img))
train_dataloader = torch.nn.utils.rnn.pad_sequence(img_tensors, batch_first=True)
print(train_dataloader.shape) # (sample_size,frames,height,width,channel)
N,T,H,W,C = train_dataloader.shape

# train_dataloader = train_dataloader.reshape(N,T,C,H,W) --incorrect shaping!!
train_dataloader = train_dataloader.permute(0,1,4,2,3) # correct shaping
print(train_dataloader.shape)

100%|██████████| 318/318 [00:06<00:00, 49.86it/s]


In [ ]:
def show_image(img):
  img = np.float32(img.detach().numpy().astype(int)/255.0)
  print(img.shape)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img)

# show frame example
show_image(train_dataloader[0][0].permute(1,2,0))

In [ ]:
# Labels
label_archive = zipfile.ZipFile('MU3D_dataset.zip', 'r')
for file in tqdm(label_archive.infolist()):
  if file.filename.endswith('.xlsx'):
    with label_archive.open(file) as label_file:
      label = pd.read_excel(label_file, sheet_name='Video-Level Data')
      # display(label.head(3))

y = torch.tensor(label['Veracity'].astype('float').values)
y = y[:61]
y = y.unsqueeze(1)
print(y.shape)

### Train / Test Split

In [ ]:
print(N,T,H,W,C)

In [ ]:
# training and testing splt
X_train, X_test, y_train, y_test = train_test_split(train_dataloader, y, test_size=0.1, random_state=42)

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

### Model Training

In [ ]:
# ViViT model
FRAME_NUM = T
model = ViViT(image_size_w=W, image_size_h=H,
                   patch_size_w=80, patch_size_h=60,
                     num_classes=1,
                     num_frames=FRAME_NUM, pool='cls').to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-5)
criterion = nn.BCELoss()

Trainer = VIVIT_Trainer(model, optimizer, criterion)
# Trainer.train(X_train, X_test, y_train, y_test, BATCH_SIZE=4, num_epoch=1)

### Classification Interpretabiity
- Attentionn weight extraction
- Emotions analysis on 6 emotions: <br>
{0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"} <br>
Using pretrained model from source: https://github.com/Dipeshtamboli/emotion-detection-using-TF

In [ ]:
from Inference import VIVIT_Classifier

X_sample = X_test[0].unsqueeze(0).float()
y_sample = y_test[0].float()
print('Input size: ',X_sample.shape)

lie_detector = VIVIT_Classifier(model)
lie_detector.inference(X_sample, y_sample)
lie_detector.extract_attn_weights(n=4)

In [ ]:
%tb